In [2]:
import os
import pandas as pd
import scipy.io
import plotly.express as px
import random
path = "/Users/jorgerag/Documents/UCSD/courses/Winter23/COGS181/final_project/data/raw"
os.chdir(path)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [3]:
# get filename info
def get_filename_info(df, filename):
    filename_info = filename.split("_")
    filename_info_sub = filename_info[2].split("-")
    df['subject'] = filename_info[1].strip()
    df['threshold'] = int(filename_info_sub[0].strip())
    df['day'] = filename_info_sub[1].replace(".mat", "").strip()

    return df

In [4]:
# get behavioral information
def get_beh(mat, filename):
    beh_data = mat['GCAMP']['beh_data'][0][0]
    beh_data = pd.DataFrame(beh_data)

    new_header = beh_data.iloc[0] #grab the first row for the header
    beh_data = beh_data[1:] #take the data less the header row
    beh_data.columns = new_header

    beh_data = get_filename_info(beh_data, filename)

    return beh_data


In [5]:
# get photometry information
def get_photo(mat, filename):
    photo_data = pd.DataFrame()
    photo_data['timestamp'] = pd.DataFrame(mat['GCAMP']['raw_gcamptimestamps'][0][0])
    photo_data['gcamp'] = pd.DataFrame(mat['GCAMP']['raw_gcampdata'][0][0])

    photo_data = get_filename_info(photo_data, filename)

    return photo_data

In [6]:
behavioral_data = pd.DataFrame()
photometry_data = pd.DataFrame()
for file in os.listdir(os.curdir):
    mat = scipy.io.loadmat(os.path.join(os.curdir, file))
    print(file)
    # behavioral dataframe
    beh_data = get_beh(mat, file)
    behavioral_data = pd.concat([behavioral_data, beh_data])
    # photometry dataframe
    photo_data = get_photo(mat, file)
    photometry_data = pd.concat([photometry_data, photo_data])

behavioral_data.columns = ["timestamp", "lp", "he", "rew", "subject", "threshold", "day"]
behavioral_data = behavioral_data.drop_duplicates()
photometry_data = photometry_data.drop_duplicates()


GCAMP_4268_1600-7.mat
GCAMP_3316_1600-5.mat
GCAMP_3316_1600-4.mat
GCAMP_4268_1600-6.mat
GCAMP_4268_1600-4.mat
GCAMP_3316_1600-6.mat
GCAMP_4268_1600-5.mat
GCAMP_4268_1600-1.mat
GCAMP_3316_1600-3.mat
GCAMP_3316_1600-2.mat
GCAMP_4268_1600-2.mat
GCAMP_4268_1600-3.mat
GCAMP_4269_1600-8.mat
GCAMP_3317_1600-3.mat
GCAMP_3203_1600-3.mat
GCAMP_4269_1600-1.mat
GCAMP_3203_1600-2.mat
GCAMP_4269_1600-3.mat
GCAMP_3317_1600-1.mat
GCAMP_3317_1600-5.mat
GCAMP_4269_1600-7.mat
GCAMP_4269_1600-6.mat
GCAMP_4269_1600-5.mat
GCAMP_3201_1600-2.mat
GCAMP_3315_1600-3.mat
GCAMP_3204_1600-6.mat
GCAMP_3204_1600-4.mat
GCAMP_3204_1600-5.mat
GCAMP_4268_1600-8.mat
GCAMP_3201_1600-4.mat
GCAMP_3201_1600-5.mat
GCAMP_3315_1600-6.mat
GCAMP_3201_1600-6.mat


In [49]:
behavioral_data[300:]

,timestamp,lp,he,rew,subject,threshold,day
322,48157665.8688,0.0000,0.0000,0.0000,4268,1600,7
323,48157665.8816,0.0000,0.0000,0.0000,4268,1600,7
324,48157669.8752,0.0000,0.0000,0.0000,4268,1600,7
325,48157674.0096,0.0000,0.0000,0.0000,4268,1600,7
326,48157678.1440,0.0000,0.0000,0.0000,4268,1600,7
...,...,...,...,...,...,...,...
1840598,51490195.6864,0.0000,0.0000,0.0000,3201,1600,6
1840599,51490195.6992,0.0000,0.0000,0.0000,3201,1600,6
1840600,51490199.8464,0.0000,0.0000,0.0000,3201,1600,6
1840601,51490203.9424,0.0000,0.0000,0.0000,3201,1600,6


In [32]:
# Plot behavior for one mice for one day
plot_beha = behavioral_data[(behavioral_data['subject']=='4268') & (behavioral_data['day']=='7')]
len(plot_beha)
fig = px.scatter(plot_beha[100000:110000], x='timestamp', y="lp")
fig.show()

In [108]:
plot_beha['timestamp'] - plot_beha['timestamp'].shift(1)

1            NaN
2         3.9936
3         4.1216
5         4.1216
6         4.0064
           ...  
1779580   4.1216
1779581   4.1216
1779583   4.6592
1779584   3.5968
1779585   4.0064
Name: timestamp, Length: 1675688, dtype: float64

In [45]:
photometry_data

,timestamp,gcamp,subject,threshold,day
12,48157641.7536,33182,4268,1600,7
13,48157666.7648,15604,4268,1600,7
14,48157691.8912,33026,4268,1600,7
15,48157717.5168,15492,4268,1600,7
16,48157741.9008,33089,4268,1600,7
...,...,...,...,...,...
224851,51490212.4544,31987,3201,1600,6
224852,51490236.8256,17865,3201,1600,6
224853,51490261.8752,32205,3201,1600,6
224854,51490286.8352,17689,3201,1600,6


In [112]:
# Plot photmetry for one mice for one day
plot_photo = photometry_data[(photometry_data['subject']=='4268') & (photometry_data['day']=='7')]
len(plot_photo)
fig = px.line(plot_photo[0:20], x='timestamp', y="gcamp")
fig.show()

In [99]:
plot_photo['timestamp'][-1:] - plot_photo['timestamp'][0]

217181   5439114.3552
Name: timestamp, dtype: float64

In [107]:
plot_photo['timestamp'] - plot_photo['timestamp'].shift(1)

0            NaN
1        24.7040
2        26.2912
3        24.3072
4        24.5248
           ...  
217177   25.3312
217178   25.1520
217179   24.6400
217180   25.1264
217181   25.0112
Name: timestamp, Length: 217182, dtype: float64

## Wav2Vec


In [7]:
model_checkpoint = "facebook/wav2vec2-base"

In [8]:
import transformers
from transformers import AutoFeatureExtractor
import numpy as np

In [9]:
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

/Users/jorgerag/.pyenv/versions/3.9.16/envs/cogs181/lib/python3.9/site-packages/transformers/configuration_utils.py:375: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [10]:
feature_extractor(np.array(photometry_data['gcamp'][0:1000]), sampling_rate=16000, max_length=100, truncation=True)['input_values'][0]

array([ 0.99451294, -0.99887243,  0.99759425, -0.99864419,  0.99873548,
       -1.00697516,  1.00912067, -1.0310551 ,  1.01927761, -0.98723189,
        1.00809357, -1.01519201,  1.01083252, -0.9952205 ,  0.99302934,
       -1.00800227,  1.00021908, -0.99590524,  1.00044733, -0.98415057,
        1.00969129, -1.00183963,  0.99097512, -1.0146214 ,  0.99554004,
       -1.00332323,  0.99953434, -1.00514919,  1.01756577, -1.03345168,
        0.9590207 , -0.9773032 ,  1.01117488, -1.01599087,  0.9893774 ,
       -0.96372257,  0.99348583, -0.99419339,  1.02190244, -0.99213918,
        1.00364277, -0.97536311,  0.99439881, -0.99795945,  0.98253003,
       -1.00298086,  1.01688103, -0.99967129,  0.9868667 , -1.01644736,
        0.96986238, -1.01427903,  1.00272979, -1.00149726,  1.02498376,
       -0.98962847,  1.00101794, -0.9897426 ,  0.98732319, -1.00823051,
        0.9759109 , -1.01724622,  1.0102619 , -0.9867754 ,  0.99816487,
       -0.99168269,  1.00387102, -0.9812975 ,  0.99862136, -0.98

## Join analog and photometry

In [11]:
# Duration of lever press
def get_duration_lp(beh_dict):
    start_time = 0
    for i in range(0, len(beh_dict)):
        if i != len(beh_dict)-1:
            if beh_dict[i]['lp'] == 1 and start_time == 0:
                start_time = beh_dict[i]['timestamp']
            elif beh_dict[i+1]['lp'] == 0 and start_time != 0:
                beh_dict[i]['lp_start_time'] = start_time
                beh_dict[i]['lp_end_time'] = beh_dict[i+1]['timestamp']
                beh_dict[i]['lp_duration'] = beh_dict[i+1]['timestamp'] - start_time - 20 # correction found in the matlab code
                start_time = 0
    return beh_dict

# Get successful lp
def met_lp(x):
    if x["lp_duration"] >= x["threshold"]:
        return 1
    else:
        return 0

# Cleaning lp_duration
def lp_duration_clean(x):
    if x["lp_duration"] < 0:
        return 0
    else:
        return x["lp_duration"]

# Get gcamp between lever presses
def get_gcamp_vector(model_dict):
    for i in range(0, len(model_dict)):
        if i == 0:
            gcamp_df = photometry_data[photometry_data["timestamp"] < model_dict[i]["timestamp"]]
        else:
            gcamp_df = photometry_data[(photometry_data["timestamp"] <= model_dict[i]["timestamp"]) & (model_dict[i-1]["timestamp"] < photometry_data["timestamp"])]
        model_dict[i]["gcamp_lp"] = feature_extractor(np.array(gcamp_df["gcamp"]), max_length=100, sampling_rate=16000, truncation=True, padding='max_length')['input_values'][0]
    return model_dict

In [12]:
# Get unique subject, threshold and day combinations
unique_obj = behavioral_data.groupby(["subject", "threshold", "day"]).size().reset_index()
unique_obj = unique_obj.to_dict('records')

# Iterate over every sesion to create final dataframe
final_model_data = pd.DataFrame()
for elem in unique_obj:
    beh_df = behavioral_data[(behavioral_data['subject'] == elem["subject"]) & (behavioral_data['threshold'] == elem["threshold"]) & (behavioral_data['day'] == elem["day"])]
    photo_df = photometry_data[(photometry_data['subject'] == elem["subject"]) & (photometry_data['threshold'] == elem["threshold"]) & (photometry_data['day'] == elem["day"])]

    beh_dict = beh_df.to_dict('records')
    beh_dict = get_duration_lp(beh_dict)

    model_df = pd.DataFrame(beh_dict)
    
    # Filter everything but lp 
    model_df = model_df.loc[pd.notna(model_df['lp_duration'])]
    # LP met
    model_df['lp_duration'] = model_df.apply(lp_duration_clean, axis = 1)
    model_df["lp_met"] = model_df.apply(met_lp, axis=1)

    model_dict = model_df.to_dict('records')
    #model_dict = get_gcamp_total(model_dict)
    model_dict = get_gcamp_vector(model_dict)
    model_df = pd.DataFrame(model_dict)
    model_df['order'] = range(1, len(model_df) + 1)
    model_df = model_df[["order", "subject", "threshold", "day", "lp_duration", "lp_met", "gcamp_lp"]]
    final_model_data = pd.concat([final_model_data, model_df])

In [13]:
final_model_data

,order,subject,threshold,day,lp_duration,lp_met,gcamp_lp
0,1,3201,1600,2,668.1536,0,"[0.99451293532181, -0.9988724315727613, 0.9975..."
1,2,3201,1600,2,668.1536,0,"[-0.980229255445724, 1.1326303506950752, -0.97..."
2,3,3201,1600,2,749.6000,0,"[1.0172488034297413, -0.9977053450551177, 1.00..."
3,4,3201,1600,2,1450.0288,0,"[-0.9966975375070194, 0.9976137767678932, -0.9..."
4,5,3201,1600,2,771.0784,0,"[-0.9934138134549483, 1.1337681918772098, -0.9..."
...,...,...,...,...,...,...,...
187,188,4269,1600,8,1847.8144,1,"[-0.998850638984861, 1.0065643226909986, -0.98..."
188,189,4269,1600,8,1769.6576,1,"[-0.9932717332773531, 1.0046274210680493, -0.9..."
189,190,4269,1600,8,1368.4160,0,"[1.015412930204221, -1.0005916681356672, 1.015..."
190,191,4269,1600,8,475.6416,0,"[-0.9513113022713279, 1.3370446278776906, -0.9..."


In [15]:
#final_model_data.to_csv("../processed/model_data.csv", index=False)
final_model_data.reset_index(inplace=True)
final_model_data.to_json("../processed/model_data.json")

## Train/test split

In [16]:
unique_obj_df = pd.DataFrame(unique_obj)[['subject', 'day']]

In [17]:
unique_obj_df

,subject,day
0,3201,2
1,3201,4
2,3201,5
3,3201,6
4,3203,2
5,3203,3
6,3204,4
7,3204,5
8,3204,6
9,3315,3


In [23]:
# Split to train and test
## Take one trial per subject randomly as test, the rest will be for training
random.seed(123)
test_trials = unique_obj_df.groupby('subject').apply(lambda x: x.sample(1)).reset_index(drop=True)
test_trials

,subject,day
0,3201,4
1,3203,3
2,3204,5
3,3315,6
4,3316,3
5,3317,5
6,4268,1
7,4269,3


In [24]:
#Splitting 
keys = list(test_trials.columns.values)
i1 = final_model_data.set_index(keys).index
i2 = test_trials.set_index(keys).index

train_df = final_model_data[~i1.isin(i2)]
test_df = final_model_data[i1.isin(i2)]


In [25]:
len(train_df) + len(test_df) == len(final_model_data)

True

In [26]:
len(test_df)/len(final_model_data)

0.1756953092569531

In [27]:
len(train_df)/len(final_model_data)

0.8243046907430469

In [28]:
len(test_df)

1693

In [29]:
#train_df.to_csv("../processed/train_data.csv", index=False)
#test_df.to_csv("../processed/test_data.csv", index=False)
train_df.to_json("../processed/train_data.json")
test_df.to_json("../processed/test_data.json")